In [100]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
! pip install sunpy[all]
from IPython.display import clear_output
clear_output()

In [102]:
import warnings

warnings.filterwarnings("ignore")

# predicting coronal mass ejections using machine learning methods

A Coronal Mass Ejection (CME) throws magnetic flux and plasma from the Sun into interplanetary space. These eruptions are actually related to solar flares -- in fact, CMEs and solar flares are considered “a single magnetically driven event” ([Webb & Howard 2012](http://adsabs.harvard.edu/abs/2012LRSP....9....3W)), wherein a flare unassociated with a CME is called a confined or compact flare. <br>

In general, the more energetic a flare, the more likely it is to be associated with a CME ([Yashiro et al. 2005](http://adsabs.harvard.edu/abs/2005JGRA..11012S05Y)) -- but this is not, by any means, a rule. For example, [Sun et al. (2015)](http://adsabs.harvard.edu/abs/2015ApJ...804L..28S) found that the largest active region in the last 24 years, shown below, produced 6 X-class flares but not a single observed CME.<br>

In this notebook, we will be predicting whether or not a flaring active region will also emit a CME using a machine learning algorithm from the scikit-learn package called Support Vector Machine.

The analysis that follows is published in [Bobra & Ilonidis, 2016, <i> Astrophysical Journal</i>, 821, 127](http://adsabs.harvard.edu/abs/2016ApJ...821..127B). If you use any of this code, we ask that you cite Bobra & Ilonidis (2016).

To do this analysis, we'll look at every active region observed by the Helioseismic and Magnetic Imager instrument on NASA's Solar Dynamics Observatory (SDO) satellite over the last eight years. Each active region is characterized by a bunch of features. These features describe the magnetic field at the solar surface. One feature, for example, is the total energy contained within an active region. Another is the total flux through an active region. We have 18 features, all of which are calculated every 12 minutes throughout an active region's lifetime. See [Bobra et al., 2014](http://link.springer.com/article/10.1007%2Fs11207-014-0529-3) for more information on how we calculate these features. <br>

We'll then ascribe each active region to one of two classes:

1. The positive class contains flaring active regions that did produce a CME.
2. The negative class contains flaring active regions that did not produce a CME.

First, we'll import some modules.

In [103]:
import numpy as np
import matplotlib.pylab as plt
import matplotlib.mlab as mlab
import pandas as pd
import scipy.stats
import requests
import urllib
import json
from datetime import datetime as dt_obj
from datetime import timedelta
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sunpy.time import TimeRange
from sunpy.net import Fido, attrs as a

pd.set_option('display.max_rows', 500)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Now we'll gather the data. The data come from three different places:

1. CME data from SOHO/LASCO and STEREO/SECCHI coronographs, which can be accesed from the [DONKI database](http://kauai.ccmc.gsfc.nasa.gov/DONKI/) at NASA Goddard. This tells us if an active region has produced a CME or not.
2. Flare data from the GOES flare catalog at NOAA, which can be accessed with the `sunpy.instr.goes.get_event_list()` function. This tells us if an active region produced a flare or not.
3. Active region data from the Solar Dynamics Observatory's Heliosesmic and Magnetic Imager instrument, which can be accessed from the [JSOC database](http://jsoc.stanford.edu/) via a JSON API. This gives us the features characterizing each active region.

### step 1: gathering data for the positive class

Let's first query the [DONKI database](http://kauai.ccmc.gsfc.nasa.gov/DONKI/) to get the data associated with the positive class. Be forewarned: there's a lot of data cleaning involved with building the positive class.

In [104]:
# request the data
baseurl = "https://kauai.ccmc.gsfc.nasa.gov/DONKI/WS/get/FLR?"
t_start = "2010-05-01"
t_end = "2024-12-31"
url = baseurl+"startDate="+t_start+"&endDate="+t_end

# if there's no response at this time, print warning
response = requests.get(url)
if response.status_code != 200:
    print('cannot successfully get an http response')

In [105]:
# read the data

print("Getting data from", url)
df = pd.read_json(url)

# select flares associated with a linked event (SEP or CME), and
# select only M or X-class flares
events_list = df.loc[df['classType'].str.contains("M|X") & ~df['linkedEvents'].isnull()]

# drop all rows that don't satisfy the above conditions
events_list = events_list.reset_index(drop=True)

Getting data from https://kauai.ccmc.gsfc.nasa.gov/DONKI/WS/get/FLR?startDate=2010-05-01&endDate=2024-12-31


In [106]:
# drop the rows that aren't linked to CME events
for i in range(events_list.shape[0]):
    value = events_list.loc[i]['linkedEvents'][0]['activityID']
    if not "CME" in value:
        print(value, "not a CME, dropping row")
        events_list = events_list.drop([i])
events_list = events_list.reset_index(drop=True)

2011-08-09T08:40:00-SEP-001 not a CME, dropping row
2011-09-07T06:00:00-SEP-001 not a CME, dropping row
2011-09-24T20:45:00-SEP-001 not a CME, dropping row
2013-06-21T06:09:00-SEP-001 not a CME, dropping row
2015-03-06T19:20:00-SEP-001 not a CME, dropping row
2015-06-18T09:25:00-SEP-001 not a CME, dropping row
2015-06-21T20:35:00-SEP-001 not a CME, dropping row
2015-06-21T20:35:00-SEP-001 not a CME, dropping row
2023-07-18T01:00:00-SEP-002 not a CME, dropping row
2023-08-05T10:00:00-SEP-001 not a CME, dropping row
2024-01-22T13:00:00-SEP-001 not a CME, dropping row
2024-05-10T14:50:00-SEP-001 not a CME, dropping row
2024-09-09T20:55:00-SEP-002 not a CME, dropping row
2024-12-21T16:12:00-SEP-001 not a CME, dropping row
2024-12-21T16:12:00-SEP-001 not a CME, dropping row
2024-12-21T16:12:00-SEP-001 not a CME, dropping row


Convert the `peakTime` column in the `events_list` dataframe from a string into a datetime object:

In [107]:
def parse_tai_string(tstr):
    year = int(tstr[:4])
    month = int(tstr[5:7])
    day = int(tstr[8:10])
    hour = int(tstr[11:13])
    minute = int(tstr[14:16])
    return dt_obj(year, month, day, hour, minute)


for i in range(events_list.shape[0]):
    events_list['peakTime'].iloc[i] = parse_tai_string(events_list['peakTime'].iloc[i])

Check for Case 1: In this case, the CME and flare exist but NOAA active region number does not exist in the DONKI database.

In [108]:
# Case 1: CME and Flare exist but NOAA active region number does not exist in DONKI database

number_of_donki_mistakes = 0  # count the number of DONKI mistakes
# create an empty array to hold row numbers to drop at the end
event_list_drops = []

for i in range(events_list.shape[0]):
    if (np.isnan(events_list.loc[i]['activeRegionNum'])):
        time = events_list['peakTime'].iloc[i]
        time_range = TimeRange(time, time)
        listofresult = Fido.search(a.Time(time_range),a.hek.EventType("FL"),a.hek.FL.GOESCls >= "M1.0",a.hek.OBS.Observatory == "GOES")

        if len(listofresult["hek"]) == 0:
            print(events_list.loc[i]['classType'], "has no match in the GOES flare database ; dropping row.")
            event_list_drops.append(i)
            number_of_donki_mistakes += 1
            continue
        else:
            if (listofresult[0]['ar_noaanum'] == 0):
                print(events_list.loc[i]['activeRegionNum'], events_list.loc[i]
                    ['classType'], "has no match in the GOES flare database ; dropping row.")
                event_list_drops.append(i)
                number_of_donki_mistakes += 1
                continue
            else:
                print("Missing NOAA number:", events_list['activeRegionNum'].iloc[i], events_list['classType'].iloc[i],
                    events_list['peakTime'].iloc[i], "should be", listofresult[0]['ar_noaanum'][0], "; changing now.")
                events_list['activeRegionNum'].iloc[i] = listofresult[0]['ar_noaanum']
                number_of_donki_mistakes += 1

# Drop the rows for which there is no active region number in both the DONKI and GOES flare databases
events_list = events_list.drop(event_list_drops)
events_list = events_list.reset_index(drop=True)
print('There are', number_of_donki_mistakes, 'DONKI mistakes so far.')

Missing NOAA number: nan X1.4 2011-09-22 11:01:00 should be 11302 ; changing now.
Missing NOAA number: nan X1.3 2012-03-07 01:14:00 should be 11430 ; changing now.
Missing NOAA number: nan M6.3 2012-03-09 03:53:00 should be 11429 ; changing now.
Missing NOAA number: nan M5.1 2012-05-17 01:47:00 should be 11476 ; changing now.
Missing NOAA number: nan X1.1 2012-07-06 23:08:00 should be 11515 ; changing now.
Missing NOAA number: nan M6.2 2012-07-28 20:56:00 should be 11532 ; changing now.
Missing NOAA number: nan M1.7 2012-11-08 02:23:00 should be 11611 ; changing now.
Missing NOAA number: nan M1.2 2013-03-15 06:58:00 should be 11692 ; changing now.
Missing NOAA number: nan X1.6 2013-05-13 02:17:00 should be 11748 ; changing now.
Missing NOAA number: nan X2.8 2013-05-13 16:05:00 should be 11748 ; changing now.
Missing NOAA number: nan X3.2 2013-05-14 01:11:00 should be 11748 ; changing now.
Missing NOAA number: nan X1.2 2013-05-15 01:48:00 should be 11748 ; changing now.
Missing NOAA num

Now we grab all the data from the GOES database in preparation for checking Cases 2 and 3.

In [109]:
# Grab all the data from the GOES database
t_start = "2010-05-01"
t_end = "2024-12-31"
time_range = TimeRange(t_start, t_end)
listofresults = Fido.search(a.Time(time_range),a.hek.EventType("FL"),a.hek.FL.GOESCls >= "M1.0",a.hek.OBS.Observatory == "GOES")
print('Grabbed all the GOES data; there are', len(listofresults["hek"]), 'events.')

Grabbed all the GOES data; there are 2302 events.


Check for Case 2: In this case, the NOAA active region number is wrong in the DONKI database.

In [110]:
# Case 2: NOAA active region number is wrong in DONKI database

# collect all the peak flares times in the NOAA database
peak_times_noaa = [item["event_peaktime"] for item in listofresults["hek"]]

for i in range(events_list.shape[0]):
    # check if a particular DONKI flare peak time is also in the NOAA database
    peak_time_donki = events_list['peakTime'].iloc[i]
    if peak_time_donki in peak_times_noaa:
        index = peak_times_noaa.index(peak_time_donki)
    else:
        continue
    # ignore NOAA active region numbers equal to zero
    if (listofresults["hek"][index]['ar_noaanum'] == 0):
        continue
    # if yes, check if the DONKI and NOAA active region numbers match up for this peak time
    # if they don't, flag this peak time and replace the DONKI number with the NOAA number
    if (listofresults["hek"][index]['ar_noaanum'] != int(events_list['activeRegionNum'].iloc[i])):
        print('Messed up NOAA number:', int(events_list['activeRegionNum'].iloc[i]), events_list['classType'].iloc[i],
              events_list['peakTime'].iloc[i], "should be", listofresults["hek"][index]['ar_noaanum'], "; changing now.")
        events_list['activeRegionNum'].iloc[i] = listofresults["hek"][index]['ar_noaanum']
        number_of_donki_mistakes += 1
print('There are', number_of_donki_mistakes, 'DONKI mistakes so far.')

Messed up NOAA number: 11943 X1.2 2014-01-07 18:32:00 should be 11944 ; changing now.
Messed up NOAA number: 12051 M1.2 2014-05-07 16:29:00 should be 12055 ; changing now.
Messed up NOAA number: 12160 M1.4 2014-07-01 11:23:00 should be 12106 ; changing now.
Messed up NOAA number: 12282 M2.4 2015-02-09 23:35:00 should be 12280 ; changing now.
Messed up NOAA number: 12321 M1.1 2015-04-23 10:07:00 should be 12322 ; changing now.
Messed up NOAA number: 12565 M7.6 2016-07-23 05:16:00 should be 12567 ; changing now.
Messed up NOAA number: 12565 M5.5 2016-07-23 05:31:00 should be 12567 ; changing now.
Messed up NOAA number: 13191 M6.0 2023-01-15 03:42:00 should be 13188 ; changing now.
Messed up NOAA number: 13312 M1.9 2023-05-22 13:37:00 should be 13314 ; changing now.
Messed up NOAA number: 13561 M1.0 2024-01-23 13:08:00 should be 13559 ; changing now.
Messed up NOAA number: 13615 M9.4 2024-03-30 21:16:00 should be 13515 ; changing now.
Messed up NOAA number: 13638 M1.0 2024-04-19 13:06:00 

Check for Case 3: In this case, the flare peak time is wrong in the DONKI database.

In [111]:
# Case 3: The flare peak time is wrong in the DONKI database.

# create an empty array to hold row numbers to drop at the end
event_list_drops = []

active_region_numbers_noaa = [item["ar_noaanum"]
                              for item in listofresults["hek"]]
flare_classes_noaa = [item["fl_goescls"] for item in listofresults["hek"]]

for i in range(events_list.shape[0]):
    # check if a particular DONKI flare peak time is also in the NOAA database
    peak_time_donki = events_list['peakTime'].iloc[i]
    if not peak_time_donki in peak_times_noaa:
        active_region_number_donki = int(
            events_list['activeRegionNum'].iloc[i])
        flare_class_donki = events_list['classType'].iloc[i]
        flare_class_indices = [i for i, x in enumerate(
            flare_classes_noaa) if x == flare_class_donki]
        active_region_indices = [i for i, x in enumerate(
            active_region_numbers_noaa) if x == active_region_number_donki]
        common_indices = list(
            set(flare_class_indices).intersection(active_region_indices))
        if common_indices:
            print("Messed up time:", int(events_list['activeRegionNum'].iloc[i]), events_list['classType'].iloc[i],
                  events_list['peakTime'].iloc[i], "should be", peak_times_noaa[common_indices[0]], "; changing now.")
            events_list['peakTime'].iloc[i] = peak_times_noaa[common_indices[0]]
            number_of_donki_mistakes += 1
        if not common_indices:
            print("DONKI flare peak time",
                  events_list['peakTime'].iloc[i], "has no match; dropping row.")
            event_list_drops.append(i)
            number_of_donki_mistakes += 1

# Drop the rows for which the NOAA active region number and flare class associated with
# the messed-up flare peak time in the DONKI database has no match in the GOES flare database
events_list = events_list.drop(event_list_drops)
events_list = events_list.reset_index(drop=True)

# Create a list of corrected flare peak times
peak_times_donki = [events_list['peakTime'].iloc[i]
                    for i in range(events_list.shape[0])]

print('There are', number_of_donki_mistakes, 'DONKI mistakes so far.')

Messed up time: 11429 X1.1 2012-03-05 04:05:00 should be 2012-03-05 04:09:00.000 ; changing now.
DONKI flare peak time 2012-03-10 17:27:00 has no match; dropping row.
Messed up time: 11745 M5.0 2013-05-22 13:38:00 should be 2013-05-22 13:32:00.000 ; changing now.
DONKI flare peak time 2014-02-09 16:14:00 has no match; dropping row.
Messed up time: 12127 M1.5 2014-08-01 18:12:00 should be 2014-08-01 18:13:00.000 ; changing now.
Messed up time: 12146 M2.0 2014-08-25 15:10:00 should be 2014-08-25 15:11:00.000 ; changing now.
DONKI flare peak time 2014-09-03 13:53:00 has no match; dropping row.
DONKI flare peak time 2014-09-09 00:28:00 has no match; dropping row.
Messed up time: 12172 M2.3 2014-09-23 23:15:00 should be 2014-09-23 23:16:00.000 ; changing now.
Messed up time: 12242 X1.8 2014-12-20 00:24:00 should be 2014-12-20 00:28:00.000 ; changing now.
Messed up time: 12445 M1.9 2015-11-04 03:25:00 should be 2015-11-04 03:26:00.000 ; changing now.
Messed up time: 12473 M2.3 2015-01-02 00:

This is our final table of events that fall into the positive class:

In [112]:
events_list

,flrID,catalog,instruments,beginTime,peakTime,endTime,classType,sourceLocation,activeRegionNum,note,submissionTime,versionId,link,linkedEvents
0,2011-02-15T01:44:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES15: SEM/XRS 1.0-8.0'}],2011-02-15T01:44Z,2011-02-15 01:56:00,2011-02-15T02:06Z,X2.2,S20W10,11158.0,,2015-07-16T19:24Z,2,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2011-02-15T02:25:00-CME-001'}]
1,2011-02-24T07:23:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES15: SEM/XRS 1.0-8.0'}],2011-02-24T07:23Z,2011-02-24 07:35:00,2011-02-24T07:42Z,M3.5,N14E87,11163.0,,2015-07-16T19:28Z,2,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2011-02-24T08:00:00-CME-001'}]
2,2011-03-07T13:44:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES15: SEM/XRS 1.0-8.0'}],2011-03-07T13:44Z,2011-03-07 14:30:00,2011-03-07T15:08Z,M2.0,N11E21,11166.0,,2013-07-18T13:28Z,1,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2011-03-07T14:40:00-CME-001'}]
3,2011-03-07T19:43:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES15: SEM/XRS 1.0-8.0'}],2011-03-07T19:43Z,2011-03-07 20:12:00,2011-03-07T21:40Z,M3.7,N30W48,11164.0,,2013-07-18T18:41Z,1,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2011-03-07T20:12:00-CME-001'}]
4,2011-03-08T03:37:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES15: SEM/XRS 1.0-8.0'}],2011-03-08T03:37Z,2011-03-08 03:58:00,2011-03-08T04:20Z,M1.5,S21E72,11171.0,,2013-07-18T19:11Z,1,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2011-03-08T05:00:00-CME-001'}]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,2024-12-25T04:46:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES-S: EXIS 1.0-8.0'}],2024-12-25T04:46Z,2024-12-25 04:49:00,2024-12-25T04:53Z,M4.9,S18E10,13932.0,,2024-12-25T13:40Z,1,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2024-12-25T04:48:00-CME-001'}]
519,2024-12-25T23:59:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-12-25T23:59Z,2024-12-26 00:30:00,2024-12-26T00:53Z,M3.1,N20E52,13938.0,Simultaneous flaring from AR 3938 (N20E52) and...,2024-12-26T16:51Z,2,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2024-12-26T01:25:00-CME-001'}]
520,2024-12-29T04:18:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-12-29T04:18Z,2024-12-29 04:30:00,2024-12-29T04:45Z,M2.0,S15E30,13939.0,,2024-12-29T14:21Z,1,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2024-12-29T06:24:00-CME-001'}]
521,2024-12-29T16:56:00-FLR-001,M2M_CATALOG,[{'displayName': 'GOES-P: EXIS 1.0-8.0'}],2024-12-29T16:56Z,2024-12-29 17:08:00,2024-12-29T17:20Z,M3.3,S12E23,13939.0,,2024-12-29T20:43Z,1,https://kauai.ccmc.gsfc.nasa.gov/DONKI/view/FL...,[{'activityID': '2024-12-29T18:24:00-CME-001'}]


Now let's query the JSOC database to see if there are active region parameters at the time of the flare. First read the following file to map NOAA active region numbers to HARPNUMs (a HARP, or an HMI Active Region Patch, is the preferred numbering system for the HMI active regions as they appear in the magnetic field data before NOAA observes them in white light):

In [113]:
answer = pd.read_csv('http://jsoc.stanford.edu/doc/data/hmi/harpnum_to_noaa/all_harps_with_noaa_ars.txt', sep=' ')

Now, let's determine at which time we'd like to predict CMEs. In general, many people try to predict a CME either 24 or 48 hours before it happens. We can report both in this study by setting a variable called `timedelayvariable`:

In [114]:
timedelayvariable = 40

Now, we'll convert subtract `timedelayvariable` from the GOES Peak Time and re-format the datetime object into a string that JSOC can understand:

In [115]:
t_rec = [(events_list['peakTime'].iloc[i] - timedelta(hours=timedelayvariable)).strftime('%Y.%m.%d_%H:%M_TAI') for i in range(events_list.shape[0])]

Now we can grab the SDO data from the JSOC database by executing the JSON queries. We are selecting data that satisfies several criteria: The data has to be [1] disambiguated with a version of the disambiguation module greater than 1.1, [2] taken while the orbital velocity of the spacecraft is less than 3500 m/s, [3] of a high quality, and [4] within 70 degrees of central meridian. If the data pass all these tests, they are stuffed into one of two lists: one for the positive class (called CME_data) and one for the negative class (called no_CME_data).

In [116]:
def get_the_jsoc_data(event_count, t_rec):
    """
    Parameters
    ----------
    event_count: number of events
                 int

    t_rec:       list of times, one associated with each event in event_count
                 list of strings in JSOC format ('%Y.%m.%d_%H:%M_TAI')

    """

    catalog_data = []
    classification = []

    for i in range(event_count):

        print("=====", i, "=====")
        # next match NOAA_ARS to HARPNUM
        idx = answer[answer['NOAA_ARS'].str.contains(
            str(int(listofactiveregions[i])))]

        # if there's no HARPNUM, quit
        if (idx.empty == True):
            print('skip: there are no matching HARPNUMs for',
                  str(int(listofactiveregions[i])))
            continue

        # construct jsoc_info queries and query jsoc database; we are querying for 25 keywords
        url = "http://jsoc.stanford.edu/cgi-bin/ajax/jsoc_info?ds=hmi.sharp_720s["+str(
            idx.HARPNUM.values[0])+"]["+t_rec[i]+"][? (CODEVER7 !~ '1.1 ') and (abs(OBS_VR)< 3500) and (QUALITY<65536) ?]&op=rs_list&key=USFLUX,MEANGBT,MEANJZH,MEANPOT,SHRGT45,TOTUSJH,MEANGBH,MEANALP,MEANGAM,MEANGBZ,MEANJZD,TOTUSJZ,SAVNCPP,TOTPOT,MEANSHR,AREA_ACR,R_VALUE,ABSNJZH"
        response = requests.get(url)

        # if there's no response at this time, quit
        if response.status_code != 200:
            print('skip: cannot successfully get an http response')
            continue

        # read the JSON output
        data = response.json()

        # if there are no data at this time, quit
        if data['count'] == 0:
            print('skip: there are no data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        # check to see if the active region is too close to the limb
        # we can compute the latitude of an active region in stonyhurst coordinates as follows:
        # latitude_stonyhurst = CRVAL1 - CRLN_OBS
        # for this we have to query the CEA series (but above we queried the other series as the CEA series does not have CODEVER5 in it)

        url = "http://jsoc.stanford.edu/cgi-bin/ajax/jsoc_info?ds=hmi.sharp_cea_720s["+str(
            idx.HARPNUM.values[0])+"]["+t_rec[i]+"][? (abs(OBS_VR)< 3500) and (QUALITY<65536) ?]&op=rs_list&key=CRVAL1,CRLN_OBS"
        response = requests.get(url)

        # if there's no response at this time, quit
        if response.status_code != 200:
            print('skip: failed to find CEA JSOC data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        # read the JSON output
        latitude_information = response.json()

        # if there are no data at this time, quit
        if latitude_information['count'] == 0:
            print('skip: there are no data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        CRVAL1 = float(latitude_information['keywords'][0]['values'][0])
        CRLN_OBS = float(latitude_information['keywords'][1]['values'][0])
        if (np.absolute(CRVAL1 - CRLN_OBS) > 70.0):
            print('skip: latitude is out of range for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        if ('MISSING' in str(data['keywords'])):
            print('skip: there are some missing keywords for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        print('accept NOAA Active Region number', str(int(
            listofactiveregions[i])), 'and HARPNUM', idx.HARPNUM.values[0], 'at time', t_rec[i])

        individual_flare_data = []
        for j in range(18):
            individual_flare_data.append(
                float(data['keywords'][j]['values'][0]))

        catalog_data.append(list(individual_flare_data))

        single_class_instance = [idx.HARPNUM.values[0], str(
            int(listofactiveregions[i])), listofgoesclasses[i], t_rec[i]]
        classification.append(single_class_instance)

    return catalog_data, classification

Now we prepare the data to be fed into the function:

In [117]:
listofactiveregions = list(events_list['activeRegionNum'].values.flatten())
listofgoesclasses = list(events_list['classType'].values.flatten())

And call the function:

In [118]:
positive_result = get_the_jsoc_data(events_list.shape[0], t_rec)

===== 0 =====
accept NOAA Active Region number 11158 and HARPNUM 377 at time 2011.02.13_09:56_TAI
===== 1 =====
skip: there are no data for HARPNUM 392 at time 2011.02.22_15:35_TAI
===== 2 =====
accept NOAA Active Region number 11166 and HARPNUM 401 at time 2011.03.05_22:30_TAI
===== 3 =====
accept NOAA Active Region number 11164 and HARPNUM 393 at time 2011.03.06_04:12_TAI
===== 4 =====
skip: there are no data for HARPNUM 415 at time 2011.03.06_11:58_TAI
===== 5 =====
accept NOAA Active Region number 11226 and HARPNUM 637 at time 2011.06.05_14:41_TAI
===== 6 =====
accept NOAA Active Region number 11261 and HARPNUM 750 at time 2011.08.01_21:48_TAI
===== 7 =====
accept NOAA Active Region number 11261 and HARPNUM 750 at time 2011.08.02_11:57_TAI
===== 8 =====
accept NOAA Active Region number 11283 and HARPNUM 833 at time 2011.09.06_06:38_TAI
===== 9 =====
skip: there are no data for HARPNUM 892 at time 2011.09.20_19:01_TAI
===== 10 =====
skip: latitude is out of range for HARPNUM 892 at 

Here is the number of events associated with the positive class:

In [119]:
CME_data = positive_result[0]
positive_class = positive_result[1]
print("There are", len(CME_data), "CME events in the positive class.")

There are 328 CME events in the positive class.


In [120]:
positive_df = pd.concat([pd.DataFrame(CME_data), pd.DataFrame(positive_class)], axis=1)
positive_df.columns = ["USFLUX", "MEANGBT", "MEANJZH", "MEANPOT", "SHRGT45", "TOTUSJH", "MEANGBH",
              "MEANALP", "MEANGAM", "MEANGBZ", "MEANJZD", "TOTUSJZ", "SAVNCPP", "TOTPOT",
              "MEANSHR", "AREA_ACR", "R_VALUE", "ABSNJZH", "HARPNUM", "NOAA", "Class", "Peak Time"]


In [121]:
positive_df

,USFLUX,MEANGBT,MEANJZH,MEANPOT,SHRGT45,TOTUSJH,MEANGBH,MEANALP,MEANGAM,MEANGBZ,...,SAVNCPP,TOTPOT,MEANSHR,AREA_ACR,R_VALUE,ABSNJZH,HARPNUM,NOAA,Class,Peak Time
0,1.491314e+22,113.743,0.024828,11209.890,51.814,2258.060,86.189,0.061114,60.275,120.811,...,2.084641e+13,3.938565e+23,49.817,693.729858,4.757,656.826,377,11158,X2.2,2011.02.13_09:56_TAI
1,2.876022e+22,87.909,-0.002668,10057.630,41.292,1844.730,51.010,-0.005494,48.306,92.966,...,3.731971e+12,5.619414e+23,41.177,983.458557,4.372,112.261,401,11166,M2.0,2011.03.05_22:30_TAI
2,5.469383e+22,98.148,0.009160,11304.520,36.043,4429.780,57.177,0.018372,45.547,107.479,...,4.568809e+13,1.170450e+24,39.703,2036.189453,4.930,714.109,393,11164,M3.7,2011.03.06_04:12_TAI
3,2.009648e+22,119.840,0.010197,5393.049,20.299,1507.634,57.890,0.028139,38.911,120.649,...,1.948402e+13,2.182683e+23,31.612,1106.514404,4.140,310.748,637,11226,M2.5,2011.06.05_14:41_TAI
4,2.143759e+22,108.229,0.026037,12600.230,45.364,2738.641,74.921,0.064264,55.212,114.936,...,4.235516e+13,5.939279e+23,45.344,1299.170166,4.758,924.141,750,11261,M6.0,2011.08.01_21:48_TAI
5,1.965895e+22,108.413,0.032539,12158.050,43.946,2515.457,73.623,0.079660,53.590,114.656,...,4.240666e+13,5.271646e+23,44.054,1245.250000,4.758,1062.329,750,11261,M9.3,2011.08.02_11:57_TAI
6,1.593603e+22,110.922,0.008814,9999.623,40.800,1742.216,69.561,0.022994,52.585,115.542,...,1.090947e+13,3.577484e+23,42.265,1334.543823,4.414,237.420,833,11283,X1.8,2011.09.06_06:38_TAI
7,5.826355e+22,103.231,-0.010924,8728.584,36.172,4570.458,58.056,-0.027959,47.600,105.843,...,2.713168e+13,1.103831e+24,40.021,3271.788086,4.765,1040.213,1321,11402,M8.7,2012.01.21_11:59_TAI
8,6.554430e+22,69.877,-0.006254,3518.320,10.640,2795.007,30.829,-0.018985,30.314,74.109,...,2.448820e+13,4.719349e+23,23.236,1361.035034,4.659,631.656,1321,11402,X1.8,2012.01.26_02:37_TAI
9,4.902454e+22,84.104,-0.025741,14946.160,41.654,4211.301,56.879,-0.041467,49.833,100.697,...,5.744073e+13,1.245575e+24,44.263,1566.479248,4.994,1615.252,1449,11429,X5.4,2012.03.05_08:24_TAI


### step 2: gathering data for the negative class

To gather the examples for the negative class, we only need to:

1. Query the GOES database for all the M- and X-class flares during our time of interest, and
2. Select the ones that are not associated with a CME.

In [122]:
# select peak times that belong to both classes
all_peak_times = np.array([(listofresults["hek"][i]['event_peaktime'])
                           for i in range(len(listofresults["hek"]))])

negative_class_possibilities = []
counter_positive = 0
counter_negative = 0
for i in range(len(listofresults["hek"])):
    this_peak_time = all_peak_times[i]
    if (this_peak_time in peak_times_donki):
        counter_positive += 1
    else:
        counter_negative += 1
        this_instance = [listofresults["hek"][i]['ar_noaanum'],
                         listofresults["hek"][i]['fl_goescls'], listofresults["hek"][i]['event_peaktime']]
        negative_class_possibilities.append(this_instance)
print("There are", counter_positive, "events in the positive class.")
print("There are", counter_negative, "events in the negative class.")

There are 534 events in the positive class.
There are 1768 events in the negative class.


Again, we compute times that are one day before the flare peak time and convert it into a string that JSOC can understand:

In [123]:
t_rec = np.array([(negative_class_possibilities[i][2] - timedelta(hours=timedelayvariable)).strftime('%Y.%m.%d_%H:%M_TAI') for i in range(len(negative_class_possibilities))])

And again, we query the JSOC database to see if these data are present:

In [124]:
listofactiveregions = list(negative_class_possibilities[i][0] for i in range(counter_negative))
listofgoesclasses = list(negative_class_possibilities[i][1] for i in range(counter_negative))

In [125]:
negative_result = get_the_jsoc_data(counter_negative, t_rec)

===== 0 =====
accept NOAA Active Region number 11069 and HARPNUM 8 at time 2010.05.04_01:19_TAI
===== 1 =====
skip: there are no data for HARPNUM 54 at time 2010.06.10_08:57_TAI
===== 2 =====
accept NOAA Active Region number 11079 and HARPNUM 49 at time 2010.06.11_13:39_TAI
===== 3 =====
accept NOAA Active Region number 11093 and HARPNUM 115 at time 2010.08.06_02:24_TAI
===== 4 =====
accept NOAA Active Region number 11112 and HARPNUM 211 at time 2010.10.15_03:12_TAI
===== 5 =====
skip: there are no data for HARPNUM 245 at time 2010.11.03_07:58_TAI
===== 6 =====
skip: there are no data for HARPNUM 245 at time 2010.11.03_07:58_TAI
===== 7 =====
skip: there are no data for HARPNUM 245 at time 2010.11.03_21:29_TAI
===== 8 =====
skip: latitude is out of range for HARPNUM 245 at time 2010.11.04_23:36_TAI
===== 9 =====
accept NOAA Active Region number 11149 and HARPNUM 345 at time 2011.01.26_09:03_TAI
===== 10 =====
accept NOAA Active Region number 11153 and HARPNUM 362 at time 2011.02.07_09:

Here is the number of events associated with the negative class:

In [126]:
no_CME_data = negative_result[0]
negative_class = negative_result[1]
print("There are", len(no_CME_data), "no-CME events in the negative class.")

There are 929 no-CME events in the negative class.


In [127]:
negative_df = pd.concat([pd.DataFrame(no_CME_data), pd.DataFrame(negative_class)], axis=1)
negative_df.columns = ["USFLUX", "MEANGBT", "MEANJZH", "MEANPOT", "SHRGT45", "TOTUSJH", "MEANGBH",
              "MEANALP", "MEANGAM", "MEANGBZ", "MEANJZD", "TOTUSJZ", "SAVNCPP", "TOTPOT",
              "MEANSHR", "AREA_ACR", "R_VALUE", "ABSNJZH", "HARPNUM", "NOAA", "Class", "Peak Time"]


In [128]:
negative_df

,USFLUX,MEANGBT,MEANJZH,MEANPOT,SHRGT45,TOTUSJH,MEANGBH,MEANALP,MEANGAM,MEANGBZ,...,SAVNCPP,TOTPOT,MEANSHR,AREA_ACR,R_VALUE,ABSNJZH,HARPNUM,NOAA,Class,Peak Time
0,6.939400e+20,125.115,-0.002422,1921.9510,7.935,45.165,44.381,-0.009514,28.854,124.762,...,1.452894e+11,2.991456e+21,26.418,20.265724,2.259,2.838,8,11069,M1.2,2010.05.04_01:19_TAI
1,1.389247e+21,96.434,0.004790,895.5663,0.047,58.704,27.896,0.017531,20.507,96.687,...,9.709253e+11,2.536888e+21,16.646,20.491779,0.000,10.217,49,11079,M1.0,2010.06.11_13:39_TAI
2,1.871663e+22,80.834,-0.005232,6007.7240,19.871,944.973,36.412,-0.013719,36.746,82.301,...,3.397039e+12,2.323586e+23,30.668,388.580292,3.857,152.353,115,11093,M1.0,2010.08.06_02:24_TAI
3,1.001843e+22,136.404,0.004658,3139.6580,11.998,799.280,66.220,0.015368,36.548,136.564,...,4.705427e+12,6.828978e+22,27.275,708.490906,3.619,76.296,211,11112,M2.9,2010.10.15_03:12_TAI
4,2.508143e+22,76.523,0.004171,6441.8630,19.964,1222.937,35.450,0.010306,35.760,81.753,...,9.660441e+12,3.124827e+23,28.649,597.599121,4.241,152.344,345,11149,M1.3,2011.01.26_09:03_TAI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,5.784432e+22,108.926,-0.012187,12943.1000,36.715,5823.211,64.129,-0.022953,47.044,119.420,...,2.837857e+13,1.322419e+24,40.423,3515.656738,5.023,937.616,12471,13936,M1.2,2024.12.29_01:30_TAI
925,5.759344e+22,109.431,-0.012941,12996.0900,36.548,5791.521,64.417,-0.024304,47.003,119.998,...,2.991659e+13,1.318910e+24,40.326,3508.676514,4.992,988.936,12471,13936,M1.6,2024.12.29_01:42_TAI
926,5.769258e+22,108.998,-0.012918,12953.6100,36.652,5753.623,64.397,-0.024275,47.002,119.705,...,2.709202e+13,1.316595e+24,40.263,3526.651611,4.960,988.626,12471,13936,M1.6,2024.12.29_02:24_TAI
927,5.777631e+22,109.027,-0.012967,12972.9000,36.641,5811.447,64.447,-0.024393,46.954,119.747,...,2.770511e+13,1.320399e+24,40.191,3525.758301,4.978,993.773,12471,13936,M1.7,2024.12.29_02:33_TAI


In [129]:
positive_df.to_csv(f"/content/drive/MyDrive/Research/Falconer Paper/CME prediction tss vs time/Data2024/positive{timedelayvariable}.csv")
negative_df.to_csv(f"/content/drive/MyDrive/Research/Falconer Paper/CME prediction tss vs time/Data2024/negative{timedelayvariable}.csv")